In [21]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [ ]:
df = pd.read_csv("../data/adult.csv", index_col=[0])

In [100]:
df = df.replace("?", np.nan).dropna()

In [ ]:
df = df[
    (
        (df["salary"] == "<=50K") & (df["salary K$"] <= 50) & (df["salary K$"] > 0) |
        (df["salary"] == ">50K") & (df["salary K$"] > 50)
)
]

# Task 1
Print the count of men and women in the dataset.

In [ ]:
print(
    f"Men: {df[df['sex'] == 'Male']['sex'].count()}, "
    f"Women: {df[df['sex'] == 'Female']['sex'].count()}"
)

# Task 2
Find the average age of men in dataset

In [ ]:
df[df["sex"] == "Male"]["age"].mean()

# Task 3
Get the percentage of people from Poland (native-country)

In [ ]:
nations = df["native-country"].value_counts()
polish = (nations["Poland"] / nations.sum()) * 100

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [ ]:
df.groupby("salary")["age"].agg(["mean","std"])

# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [ ]:
df[(~df["education"].isin(["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"])) & (df["salary"] == ">50K")]

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [ ]:
df.groupby("education")["age"].describe()

# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [ ]:
men = df[df["sex"] == "Male"].loc[:, ("sex", "marital-status", "salary K$")]
men["is_married"] = np.where(men["marital-status"].str.startswith("Married"), "Married", "Not_married")
men.groupby("is_married")["salary K$"].mean()

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [ ]:
max_hours_per_week = df["hours-per-week"].max()
df[df["hours-per-week"] == max_hours_per_week].shape[0]

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [ ]:
df.corr()
# The older people have the higher salary, salary depends on working hours per week